In [12]:
# imports
import collections
from importlib import reload

import displayImages as di
import polars as pl
from tqdm import tqdm

reload(di)

<module 'displayImages' from '/dgx1nas1/storage/data/jess/repos/2021_09_01_VarChAMP/7.downstream_analysis_jess/notebooks/displayImages.py'>

In [2]:
metrics_dir = "/dgx1nas1/storage/data/jess/varchamp/sc_data/classification_results/B7B8_1percent_updatedmeta"
prof_dir = "/dgx1nas1/storage/data/jess/repos/2021_09_01_VarChAMP/6.downstream_analysis_snakemake/outputs/batch_profiles"
img_dir = "/dgx1nas1/storage/data/jess/varchamp/images"
metrics_wtvar = pl.read_csv(f"{metrics_dir}/metrics_summary.csv")

In [8]:
morph = (
    metrics_wtvar.filter(pl.col("Classifier_type") == "morphology")
    .filter(pl.col("AUROC_mean") > 0.98)
    .filter(pl.col("Batch") == "batch7")
)
morph_alleles = morph.select("allele_0").to_series().unique().to_list()

In [9]:
# Get metadata required for plotting (for batch 7 only here)
pm_df = pl.scan_parquet(
    f"{prof_dir}/2024_01_23_Batch_7/profiles_tcdropped_filtered_var_mad_outlier_featselect_filtcells_metacorr.parquet"
)
meta_cols = [
    "Metadata_Well",
    "Metadata_Plate",
    "Metadata_gene_allele",
    "Metadata_node_type",
]
pm_df = pm_df.select(meta_cols).unique().collect()

pm_df = pm_df.rename({
    "Metadata_Well": "Well",
    "Metadata_Plate": "Plate",
    "Metadata_gene_allele": "Allele",
    "Metadata_node_type": "control_type",
}).with_columns(
    pl.lit("05").alias("Site"),
    pl.col("Plate").str.slice(11, 6).alias("Batch"),
    pl.col("Plate").str.slice(11, 9).alias("Plate"),
)

rep_df = pl.DataFrame({
    "Batch": [
        "B7A1R1",
        "B7A1R1",
        "B7A1R1",
        "B7A1R1",
        "B7A2R1",
        "B7A2R1",
        "B7A2R1",
        "B7A2R1",
        "B8A1R2",
        "B8A1R2",
        "B8A1R2",
        "B8A1R2",
        "B8A2R2",
        "B8A2R2",
        "B8A2R2",
        "B8A2R2",
    ],
    "Replicate": [
        "T1",
        "T2",
        "T3",
        "T4",
        "T1",
        "T2",
        "T3",
        "T4",
        "T1",
        "T2",
        "T3",
        "T4",
        "T1",
        "T2",
        "T3",
        "T4",
    ],
})

pm_df = pm_df.join(rep_df, on="Batch")

In [13]:
# Define allele list and plot dir
alleles = morph_alleles
plot_dir = f"{img_dir}/B7B8_morph"
channels = ["DAPI", "GFP", "AGP", "Mito"]

In [14]:
# plot the 5th site from all images, organized by classifier
for var_allele in tqdm(alleles):
    for channel in channels:
        img_nm = f"{var_allele}_{channel}.png"

        wt_allele = var_allele.split("_")[0]
        plot_img = (
            pm_df.filter(
                ((pl.col("Allele") == var_allele) | (pl.col("Allele") == wt_allele))
            )
            .unique()
            .sort(["Allele", "Batch", "Replicate"])
        )

        # filter to keep only images where there is a single WT-VAR pair in each plate
        plates_img = (
            plot_img.select(["Plate", "control_type"])
            .unique()
            .select("Plate")
            .to_series()
            .to_list()
        )
        plates_img = collections.Counter(plates_img)
        plates_img = (
            pl.DataFrame(plates_img)
            .melt()
            .filter(pl.col("value") == 2)
            .select("variable")
            .to_series()
            .to_list()
        )
        plot_img = plot_img.filter(pl.col("Plate").is_in(plates_img))

        di.plotMultiImages(
            plot_img, channel, 0.99, 4, display=False, plotpath=f"{plot_dir}/{img_nm}"
        )

  0%|          | 0/12 [00:00<?, ?it/s]

100%|██████████| 12/12 [12:29<00:00, 62.43s/it]


Next, will manually look at images and choose the ones that look like the have the most different morphology.